In [1]:
import torch
from attention_dynamic_model import AttentionDynamicModel, set_decode_type
from reinforce_baseline import RolloutBaseline
from train import train_model

from time import strftime, gmtime
from utils import create_data_on_disk, get_cur_time

In [2]:
# Params of model
SAMPLES = 512 # 128*10000
BATCH = 10
START_EPOCH = 0
END_EPOCH = 5
FROM_CHECKPOINT = False
embedding_dim = 128
LEARNING_RATE = 0.0001
ROLLOUT_SAMPLES = 10000
NUMBER_OF_WP_EPOCHS = 1
GRAD_NORM_CLIPPING = 1.0
BATCH_VERBOSE = 1000
VAL_BATCH_SIZE = 10
VALIDATE_SET_SIZE = 10000
SEED = 1234
GRAPH_SIZE = 50
FILENAME = 'VRP_{}_{}'.format(GRAPH_SIZE, strftime("%Y-%m-%d", gmtime()))

In [3]:
# Initialize model
model_pt = AttentionDynamicModel(embedding_dim).cuda()
set_decode_type(model_pt, "sampling")
print(get_cur_time(), 'model initialized')

2021-03-16 23:32:57 model initialized


In [4]:
# Create and save validation dataset
validation_dataset = create_data_on_disk(GRAPH_SIZE,
                                         VALIDATE_SET_SIZE,
                                         is_save=True,
                                         filename=FILENAME,
                                         is_return=True,
                                         seed = SEED)
print(get_cur_time(), 'validation dataset created and saved on the disk')

2021-03-16 23:32:57 validation dataset created and saved on the disk


In [5]:
# Initialize optimizer
optimizer = torch.optim.Adam(params=model_pt.parameters(), lr=LEARNING_RATE)

In [6]:
# Initialize baseline
baseline = RolloutBaseline(model_pt,
                           wp_n_epochs = NUMBER_OF_WP_EPOCHS,
                           epoch = 0,
                           num_samples=ROLLOUT_SAMPLES,
                           filename = FILENAME,
                           from_checkpoint = FROM_CHECKPOINT,
                           embedding_dim=embedding_dim,
                           graph_size=GRAPH_SIZE
                           )
print(get_cur_time(), 'baseline initialized')

Rollout greedy execution:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating baseline model on baseline dataset (epoch = 0)


Rollout greedy execution: 100%|██████████| 10/10 [00:05<00:00,  1.89it/s]

2021-03-16 23:33:02 baseline initialized


In [7]:
torch.cuda.empty_cache()

In [8]:
train_model(optimizer,
            model_pt,
            baseline,
            validation_dataset,
            samples = SAMPLES,
            batch = BATCH,
            val_batch_size = VAL_BATCH_SIZE,
            start_epoch = START_EPOCH,
            end_epoch = END_EPOCH,
            from_checkpoint = FROM_CHECKPOINT,
            grad_norm_clipping = GRAD_NORM_CLIPPING,
            batch_verbose = BATCH_VERBOSE,
            graph_size = GRAPH_SIZE,
            filename = FILENAME
            )

batch calculation at epoch 0: 0it [00:00, ?it/s]

Current decode type: sampling


batch calculation at epoch 0: 1it [00:00,  3.84it/s]

grad_global_norm = 0.9096242785453796, clipped_norm = 0.9096242785453796
Epoch 0 (batch = 0): Loss: -0.512176513671875: Cost: 30.47513771057129


batch calculation at epoch 0: 18it [00:03,  4.58it/s]
Rollout greedy execution:   0%|          | 0/10 [00:00<?, ?it/s]


Evaluating candidate model on baseline dataset (callback epoch = 0)


RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 5.79 GiB total capacity; 4.73 GiB already allocated; 17.75 MiB free; 4.88 GiB reserved in total by PyTorch)